In [19]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher
import json
import folium

In [20]:
df = pd.read_pickle("data/data.pkl")
df['Date'] = df['Votation'].map(lambda x : x.split(' ')[0])
df['Votation'] = df['Votation'].map(lambda x : ' '.join(x.split(' ')[1:]))
df.head()

,Commune,Votation,Electeurs inscrits,Bulletins rentrés,Participation en %,Bulletins valables,Oui,Non,Oui en %,District,Canton,Pays,Date
0,Aeugst am Albis,Initiative Droleg,1070.0,487.0,45.5,478.0,167.0,311.0,34.9,Affoltern,Zürich,Suisse,29.11.1998
1,Aeugst am Albis,Initiative sur les bourses d'études,1380.0,706.0,51.2,695.0,186.0,509.0,26.8,Affoltern,Zürich,Suisse,14.06.2015
2,Aeugst am Albis,Loi fédérale sur le renseignement,1400.0,670.0,47.9,659.0,417.0,242.0,63.3,Affoltern,Zürich,Suisse,25.09.2016
3,Aeugst am Albis,Encouragement des transports publics,835.0,321.0,38.4,312.0,128.0,184.0,41.0,Affoltern,Zürich,Suisse,03.03.1991
4,Aeugst am Albis,Réforme de l'imposition des entreprises III,1395.0,759.0,54.4,750.0,318.0,432.0,42.4,Affoltern,Zürich,Suisse,12.02.2017


In [21]:
xls = pd.ExcelFile('data/Recommandations des partis.xls')
nb_sheets = len(xls.sheet_names)
recommend = pd.DataFrame()

for i in range ((2017 - 1981 + 1)) :
#for i in range (6,7) :
 
    x = xls.parse(i)
    x.columns = range(len(x.columns))
    
    base_nb = x[x.iloc[:, 0] == 'Parti 1)'].index[0]
    
    x = x.dropna(axis=1, how='all')

    recommend_inter = pd.concat([x.iloc[base_nb:base_nb + 1], x.iloc[base_nb + 2:base_nb + 3], x.dropna()]).transpose()
    recommend_inter.iloc[0, 2:] = recommend_inter.iloc[0, 2:].map(lambda x : x.split(' ')[0])    
    
    recommend_inter.columns = range(len(recommend_inter.columns))
    
    recommend_inter.iloc[:, 0] = recommend_inter.iloc[:, 0] + str(2017 - i)
    recommend_inter.iloc[:, 1] = recommend_inter.iloc[:, 1].map(lambda x : x[0].upper() + x[1:], na_action='ignore')

    propositions = x[x.iloc[:, 0].str.contains('No ').fillna(False)]
    propositions.columns = range(len(propositions.columns))
    
    if (propositions.iloc[:, 1].isnull()).all():
        if (2017 - i == 1997):
            propositions.iloc[:, 1] = propositions[0].map(lambda x : x.split(' ')[1])
            propositions.iloc[:, 0] = propositions[0].map(lambda x : x.split(' ')[0].rstrip())
        else:
            propositions.iloc[:, 1] = propositions[0].map(lambda x : x.split(':')[1][1:])
            propositions.iloc[:, 0] = propositions[0].map(lambda x : x.split(':')[0].rstrip())
    
    dico_no_propos = propositions.dropna(1).set_index(0).to_dict()
    
    if (dico_no_propos.get(1) == None) :
        recommend_inter.iloc[:, 1] = recommend_inter.iloc[:, 1].map(dico_no_propos)
    else :
        recommend_inter.iloc[:, 1] = recommend_inter.iloc[:, 1].map(dico_no_propos[1])
        
    recommend_columns = ['Date', 'Votation'] + list(recommend_inter.iloc[0][2:])
    recommend_inter.columns = recommend_columns
    recommend_inter['Date'] = recommend_inter['Date'].fillna(method='ffill')
    recommend_inter = recommend_inter.transpose()
    
    recommend_inter = recommend_inter.groupby(recommend_inter.index).first()
    recommend = recommend.join(recommend_inter, how='outer', lsuffix='l', rsuffix='r')

recommend = recommend.transpose()  

recommend = recommend[~recommend.loc[:, 'Date'].str.contains('Parti 1').fillna(False)]
recommend.index = range(len(recommend.index))
recommend = recommend[~recommend['Votation'].isnull()]

month_to_int = { \
    'janvier' : '01', \
    'février' : '02', \
    'févirer' : '02', \
    'mars' : '03', \
    'avril' : '04', \
    'mai' : '05', \
    'juin' : '06', \
    'juillet' : '07', \
    'aout' : '08', \
    'septembre' : '09', \
    'octobre' : '10', \
    'novembre' : '11', \
    'décembre' : '12' \
}

def good_format_month (x) :
    if '.' in x :
        split_x = x.split('.')
        return split_x[0].zfill(2) + '.' + split_x[1].zfill(2) + '.' + split_x[2][:4]
    else :
        return str(x.split(' ')[0]).zfill(2) + '.' +  month_to_int[str(x.split(' ')[1]) ] + '.' + x[-4:] 

recommend.loc[:, 'Date'] = recommend.loc[:, 'Date'].map(lambda x : good_format_month(x))

recommend = recommend.loc[:, recommend.columns.drop_duplicates() ]
parties = list(recommend.columns.drop_duplicates())
parties.remove('Date')
parties.remove('Votation')
parties.remove('Parti')

recommend = recommend.loc[:, ['Date', 'Votation'] + parties]
recommend.head()

/home/david/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:601: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


,Date,Votation,AdI,DS,Lega,MCR,PBD,PCS,PDC,PES,...,PLS,POCH,PRD,PS,PSL,PST,PVL,Rep.,UDC,UDF
0,12.02.2017,Naturalisation facilitée des étrangers de la t...,NaN,NaN,-,non,oui,NaN,oui,oui,...,NaN,NaN,NaN,oui,NaN,NaN,oui,NaN,non,NaN
1,12.02.2017,Fonds pour les routes nationales et pour le tr...,NaN,NaN,-,oui,oui,NaN,oui,non,...,NaN,NaN,NaN,non,NaN,NaN,oui,NaN,oui,NaN
2,12.02.2017,Réforme de l'imposition des entreprises III,NaN,NaN,-,oui,oui,NaN,oui,non,...,NaN,NaN,NaN,non,NaN,NaN,oui,NaN,oui,NaN
3,21.05.2017,Loi sur l'énergie,NaN,NaN,-,non,oui,NaN,oui,oui,...,NaN,NaN,NaN,oui,NaN,NaN,oui,NaN,non,NaN
7,28.02.2016,Initiative sur la réforme de l'imposition du c...,NaN,NaN,NaN,NaN,non,NaN,oui,non,...,NaN,NaN,NaN,non,NaN,NaN,non,NaN,oui,NaN


In [22]:
parties

['AdI',
 'DS',
 'Lega',
 'MCR',
 'PBD',
 'PCS',
 'PDC',
 'PES',
 'PEV',
 'PLR',
 'PLS',
 'POCH',
 'PRD',
 'PS',
 'PSL',
 'PST',
 'PVL',
 'Rep.',
 'UDC',
 'UDF']

In [23]:
recommend.columns.drop_duplicates()

Index(['Date', 'Votation', 'AdI', 'DS', 'Lega', 'MCR', 'PBD', 'PCS', 'PDC',
       'PES', 'PEV', 'PLR', 'PLS', 'POCH', 'PRD', 'PS', 'PSL', 'PST', 'PVL',
       'Rep.', 'UDC', 'UDF'],
      dtype='object')

In [24]:
def translate_choice(x) :
    if (x == 1 or str(x) == 'oui') :
        return 1
    elif (x == 2 or str(x) == 'non'):
        return -1
    else :
        return 0

good_recommend = recommend.copy() 
good_recommend.loc[:, parties] = good_recommend.loc[:, parties].applymap(lambda x : translate_choice(x))
good_recommend

,Date,Votation,AdI,DS,Lega,MCR,PBD,PCS,PDC,PES,...,PLS,POCH,PRD,PS,PSL,PST,PVL,Rep.,UDC,UDF
0,12.02.2017,Naturalisation facilitée des étrangers de la t...,0,0,0,-1,1,0,1,1,...,0,0,0,1,0,0,1,0,-1,0
1,12.02.2017,Fonds pour les routes nationales et pour le tr...,0,0,0,1,1,0,1,-1,...,0,0,0,-1,0,0,1,0,1,0
2,12.02.2017,Réforme de l'imposition des entreprises III,0,0,0,1,1,0,1,-1,...,0,0,0,-1,0,0,1,0,1,0
3,21.05.2017,Loi sur l'énergie,0,0,0,-1,1,0,1,1,...,0,0,0,1,0,0,1,0,-1,0
7,28.02.2016,Initiative sur la réforme de l'imposition du c...,0,0,0,0,-1,0,1,-1,...,0,0,0,-1,0,0,-1,0,1,0
8,28.02.2016,Initiative pour le renvoi effectif des étrange...,0,0,0,0,-1,0,-1,-1,...,0,0,0,-1,0,0,-1,0,1,0
9,28.02.2016,Initiative contre la spéculation sur les denré...,0,0,0,0,-1,0,-1,1,...,0,0,0,1,0,0,-1,0,-1,0
10,28.02.2016,Réfection du tunnel routier du Gothard,0,0,0,0,1,0,1,-1,...,0,0,0,-1,0,0,-1,0,1,0
11,05.06.2016,Initiative populaire «En faveur du service pub...,0,0,0,0,-1,0,-1,-1,...,0,0,0,-1,0,0,-1,0,-1,0
12,05.06.2016,Initiative populaire «Pour un revenu de base i...,0,0,0,0,-1,0,-1,1,...,0,0,0,-1,0,0,-1,0,-1,0


In [25]:
good_recommend[good_recommend['Votation'].isnull()]

,Date,Votation,AdI,DS,Lega,MCR,PBD,PCS,PDC,PES,...,PLS,POCH,PRD,PS,PSL,PST,PVL,Rep.,UDC,UDF


In [26]:
dico_recom_vote = {}

for date1, group1 in df[df['Commune'] == 'Aeugst am Albis'].groupby('Date'):
    for date2, group2 in good_recommend.groupby('Date'):
        if (date1 == date2) :
            vot1 = list(group1['Votation']) 
            vot2 = list(group2['Votation'])
            while (len(vot1) > 0 and len(vot2) > 0) :
                max_v = -1
                max_match = [0, 0]
                for elem1  in vot1 :
                    for elem2 in  vot2 :
                        current_v = SequenceMatcher(None, elem1, elem2).ratio()
                        if(current_v > max_v) :
                            max_v = current_v
                            max_match=[elem1, elem2]
                            
                dico_recom_vote[max_match[0]] = max_match[1]
                dico_recom_vote[max_match[1]] = max_match[0]
                vot1.remove(max_match[0])
                vot2.remove(max_match[1])
            
dico_recom_vote

{"10e révision de l'AVS": "Loi fédérale sur l'assurance-vieillesse et survivants",
 "11e révision de l'AVS": "Loi fédérale sur l'assurance-vieillesse et survivants (11e révision de l'AVS)",
 "40 places d'armes ça suffit": "Initiative populaire «40 places d'armes ça suffit! - L'armée doit aussi se soumettre à la législation sur la protection de l'environnement»",
 "5e révision de l'AI": "Loi fédérale sur l'assurance-invalidité",
 '6 semaines de vacances pour tous': 'Initiative populaire «6 semaines de vacances pour tous»',
 "Abaissement de l'âge de la retraite": "Initiative populaire «visant à abaisser à 62 ans pour les hommes et à 60 ans pour les femmes l'âge donnant droit à la rente AVS»",
 "Abandon de l'énergie atomique": "Initiative populaire «pour un abandon progressif de l'énergie atomique»",
 'Abolition de la vivisection': "Initiative populaire «pour l'abolition des expériences sur animaux»",
 'Abrogation du service militaire obligatoire': "Initiative populaire «Oui à I'abrogatio

In [27]:
recommend_to_join = good_recommend.copy()
recommend_to_join['Votation'] = recommend_to_join['Votation'].map(dico_recom_vote)
recommend_to_join.head()

,Date,Votation,AdI,DS,Lega,MCR,PBD,PCS,PDC,PES,...,PLS,POCH,PRD,PS,PSL,PST,PVL,Rep.,UDC,UDF
0,12.02.2017,Naturalisation facilitée des étrangers de la t...,0,0,0,-1,1,0,1,1,...,0,0,0,1,0,0,1,0,-1,0
1,12.02.2017,Fonds pour les routes nationales et pour le tr...,0,0,0,1,1,0,1,-1,...,0,0,0,-1,0,0,1,0,1,0
2,12.02.2017,Réforme de l'imposition des entreprises III,0,0,0,1,1,0,1,-1,...,0,0,0,-1,0,0,1,0,1,0
3,21.05.2017,Loi sur l'énergie,0,0,0,-1,1,0,1,1,...,0,0,0,1,0,0,1,0,-1,0
7,28.02.2016,Initiative sur la réforme de l'imposition du c...,0,0,0,0,-1,0,1,-1,...,0,0,0,-1,0,0,-1,0,1,0


In [28]:
switzerland_coord = [46.765213, 8.252444]
town_geo_path = r'data/switzerland_borders/admin_level_8.geojson'
geo_json_data = json.load(open(town_geo_path, encoding="utf8"))
commune = [x['name'] for x in geo_json_data['features']]

to_map = df.merge(recommend_to_join.loc[:, ['Votation'] + parties], on='Votation')
to_map = to_map[to_map['Commune'].isin(commune)]
to_map.head()

,Commune,Votation,Electeurs inscrits,Bulletins rentrés,Participation en %,Bulletins valables,Oui,Non,Oui en %,District,...,PLS,POCH,PRD,PS,PSL,PST,PVL,Rep.,UDC,UDF
0,Aeugst am Albis,Initiative Droleg,1070.0,487.0,45.5,478.0,167.0,311.0,34.9,Affoltern,...,-1,0,-1,1,-1,1,0,0,-1,-1
1,Affoltern am Albis,Initiative Droleg,5729.0,2286.0,39.9,2236.0,678.0,1558.0,30.3,Affoltern,...,-1,0,-1,1,-1,1,0,0,-1,-1
2,Bonstetten,Initiative Droleg,2596.0,1063.0,40.9,1045.0,358.0,687.0,34.3,Affoltern,...,-1,0,-1,1,-1,1,0,0,-1,-1
3,Hausen am Albis,Initiative Droleg,2081.0,807.0,38.8,792.0,281.0,511.0,35.5,Affoltern,...,-1,0,-1,1,-1,1,0,0,-1,-1
4,Hedingen,Initiative Droleg,1858.0,810.0,43.6,791.0,246.0,545.0,31.1,Affoltern,...,-1,0,-1,1,-1,1,0,0,-1,-1


In [29]:
for parti in parties :
    current_to_map = to_map.loc[:, ['Commune', 'Oui en %', parti]]
    current_to_map = current_to_map[current_to_map[parti] != 0]
    current_to_map['Agreement'] = current_to_map[['Oui en %', parti]] \
        .apply(lambda x : x['Oui en %'] if x[parti] == 1 else 100 - x['Oui en %'], axis=1)
    
    current_to_map = current_to_map.groupby('Commune', as_index=False).mean()
    
    map1 = folium.Map(location=switzerland_coord, zoom_start=8)
    map1.choropleth(geo_data = geo_json_data, \
                    data = current_to_map, \
                    columns = ['Commune', 'Agreement'], \
                    key_on = 'feature.name', \
                    fill_color = 'RdYlGn', \
                    fill_opacity = 0.7, \
                    line_opacity = 0.2, \
                    legend_name = 'Agreement in % with ' + parti, \
                    threshold_scale = [0, 50, 50, 50, 50, 100])
    
    map1.save('data/maps_partis/map_' + parti + '.html')
    print(map1)